In [ ]:
import pandas as pd

pre_separated_data = True  # Setar para False quando os dados de cadastro não estiverem separados em dataframes menores

## EDA

**Dados de treino (2022)**

Este é o conjunto de dados que você e sua equipe irão usar para trabalhar, criar o modelo, fazer testes e desenvolver a solução final. 

Transações: Data, PDV, Produto, Quantidade, Faturamento. 

Cadastro de produtos: Produto, Categoria, Descrição, + até 4 atributos.

Cadastro de PDVs: PDV, On/Off Prem, Categoria (c-store, g-store, liquor etc.), Zipcode. 

In [24]:
transacoes_path = "/data/challenges/hackathon_bigdata_2025/raw_data/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet"
cadastro_prod_path = "/data/challenges/hackathon_bigdata_2025/raw_data/part-00000-tid-6364321654468257203-dc13a5d6-36ae-48c6-a018-37d8cfe34cf6-263-1-c000.snappy.parquet"
cadastro_pdv_path = "/data/challenges/hackathon_bigdata_2025/raw_data/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet"

Separando o .parquet original do cadastro de produtos, para posteriormente acessá-lo via pandas:

In [25]:
if not pre_separated_data:
    import polars as pl

    df3 = pl.read_parquet(cadastro_prod_path)
    print(df3.shape)

    total_rows = df3.shape[0]
    chunk_size = total_rows // 10  # Integer division to get the chunk size

    for i in range(10):
        start_index = i * chunk_size
        end_index = start_index + chunk_size

        if i == 9:
            chunk = df3.slice(start_index)
        else:
            chunk = df3.slice(start_index, chunk_size)

        output_filename = (
            cadastro_prod_path.strip(".snappy.parquet") + f"_part_{i + 1}.parquet"
        )

        chunk.write_parquet(output_filename)
        print(f"Written {output_filename}")

In [4]:
df_transacoes = pd.read_parquet(transacoes_path)
df_cadastro_pdv = pd.read_parquet(cadastro_pdv_path)

if pre_separated_data:
    all_dfs = []
    for i in range(10):
        df3 = pd.read_parquet(
            cadastro_prod_path.strip(".snappy.parquet") + f"_part_{i + 1}.parquet"
        )
        all_dfs.append(df3)
    df_cadastro_prod = pd.concat(all_dfs, ignore_index=True)
    del all_dfs

In [5]:
df_transacoes.shape, df_cadastro_pdv.shape, df_cadastro_prod.shape

((6560698, 11), (14419, 4), (192356316, 8))

**Identificando se há linhas duplicadas nos dataframes:**

In [ ]:
df_cadastro_pdv.drop_duplicates(inplace=True)
df_cadastro_prod.drop_duplicates(inplace=True)
df_transacoes.drop_duplicates(inplace=True)

df_cadastro_pdv.shape, df_cadastro_prod.shape, df_transacoes.shape

((14419, 4), (304956, 8), (6560698, 12))

**Identificando se há valores nulos / faltantes nos dataframes**

In [34]:
df_cadastro_pdv.isnull().sum()

pdv              0
premise          0
categoria_pdv    0
zipcode          0
dtype: int64

In [ ]:
df_cadastro_prod.isnull().sum()  # Aparentemente a coluna label não invalida o uso desses dados, já é que ela especifica uma característica categória do produto (que pode ou não ser usada).

produto             0
categoria           0
descricao           0
tipos               0
label           63339
subcategoria        0
marca               0
fabricante          0
dtype: int64

In [32]:
df_cadastro_prod

,produto,categoria,descricao,tipos,label,subcategoria,marca,fabricante
0,69753381296344216,Package,MONDAY NIGHT BREW DON'T CALL IT HOTLANTA 6/4/1...,Package,In&Out,Red Wine,Monday Night Don't Call It Hotlanta,Monday Night Brewing
1,5019314249828979377,Distilled Spirits,BARRELL V907 RED PHONE BOOTH & TOWER,Distilled Spirits,Specialty,Lager / Pilsner,Barrell Bourbon- Private Label,Barrell Craft Spirits
2,4016404282141162328,Distilled Spirits,DRINKWORKS SIMPLY REFRESHING GIN & TONIC 12/4/...,Distilled Spirits,Core,Lager,Drinkworks Simply Refreshing Gin & Tonic,AB Drinkworks
3,6217366559810422145,Distilled Spirits,CENZON REPOSADO 100% AGAVE 6/750ML 80PF,Distilled Spirits,None,Lager,Cenzon Reposado 100% Agave Tequila,Sazerac Spirits
4,7356488787409434558,Draft,REFORMATION STARK 1/4 KEG,Draft,Core,Red Wine,Reformation Stark,Reformation Brewery
...,...,...,...,...,...,...,...,...
183621172,6964240149904972585,Distilled Spirits,PAUL MASSON PEACH BRANDY 6/1.75L 54PF,Distilled Spirits,None,Ready To Drink Cocktails,Paul Masson Peach Brandy,Sazerac Spirits
183621178,5759670436535233385,Distilled Spirits,MR BOSTON BLUE CURACAO 12/1L 30PF,Distilled Spirits,None,Ready To Drink Cocktails,Mr. Boston Blue Curacao Liqueur,Sazerac Spirits
183621205,8225211493147767417,Wine,SANS WINE CO. - POOR RANCH VINEYARDS ROSE OF C...,Wine < 14 %,Discontinued,Ready To Drink Cocktails,Poor Ranch Vineyards Rose Of Carignan,Sans Wine Company
183621218,3128727580621798365,Package,MICHELOB ULTRA PRICKLY PEAR LIME 4/6/12 LN,Package,Core,Ready To Drink Cocktails,Michelob Ultra Prickly Pear Lime,AB Anheuser Busch Inc


In [ ]:
df_cadastro_prod.groupby("produto").value_counts()

produto              categoria  descricao                                    tipos        label         subcategoria      marca                                fabricante         
1000423277513436210  Draft      REFORMATION A COLD ONE 97.1 PILSNER 1/4 KEG  Draft        Core          Ale               Reformation A Cold One 97.1 Pilsner  Reformation Brewery    1
                                                                                                        Bourbon Whiskey   Reformation A Cold One 97.1 Pilsner  Reformation Brewery    1
                                                                                                        Brandy / Cognac   Reformation A Cold One 97.1 Pilsner  Reformation Brewery    1
                                                                                                        Canadian Whisky   Reformation A Cold One 97.1 Pilsner  Reformation Brewery    1
                                                                                     

In [ ]:
df_cadastro_prod[df_cadastro_prod.produto == "998529431391212014"].head(
    10
)  # Para um mesmo produto, há variações na descrição da subcategoria

,produto,categoria,descricao,tipos,label,subcategoria,marca,fabricante
65742025,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Bourbon Whiskey,Lyrarakis Assyrtiko Voila,Lyrarakis
65746450,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Other Spirits,Lyrarakis Assyrtiko Voila,Lyrarakis
65747335,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Other Wine,Lyrarakis Assyrtiko Voila,Lyrarakis
65749105,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Tequila / Mezcal,Lyrarakis Assyrtiko Voila,Lyrarakis
65751760,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Vodka,Lyrarakis Assyrtiko Voila,Lyrarakis
65755300,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Liqueurs & Cordials,Lyrarakis Assyrtiko Voila,Lyrarakis
65756185,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Ready-to-Drink,Lyrarakis Assyrtiko Voila,Lyrarakis
65772115,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Scotch Whisky,Lyrarakis Assyrtiko Voila,Lyrarakis
65777425,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Canadian Whisky,Lyrarakis Assyrtiko Voila,Lyrarakis
65811055,998529431391212014,Wine,LYRARAKIS ASSYRTIKO VOILA 2020,Wine < 14 %,Discontinued,Brandy / Cognac,Lyrarakis Assyrtiko Voila,Lyrarakis


In [ ]:
df_cadastro_prod.groupby("produto").size()

produto
1000423277513436210    43
1000794189319388082    43
1000817480846862155    43
1004943868572044494    43
1005164764874054592    43
                       ..
996135836802743128     43
997545005038320324     43
997691518031628045     43
998529431391212014     43
999285078291803499     43
Length: 7092, dtype: int64

In [ ]:
# Agrupando o dataframe pelo produto, checar quantas variações de subcategoria e categoria existem para cada produto
df_cadastro_prod.groupby("produto").agg(
    {
        "subcategoria": "nunique",
        "categoria": "nunique",
        "fabricante": "nunique",
        "label": "nunique",
    }
).reset_index().rename(
    columns={
        "subcategoria": "num_subcategorias",
        "categoria": "num_categorias",
        "fabricante": "num_fabricantes",
        "label": "num_labels",
    }
).sort_values(by=["num_subcategorias", "num_categorias"], ascending=False)

,produto,num_subcategorias,num_categorias,num_fabricantes,num_labels
0,1000423277513436210,43,1,1,1
1,1000794189319388082,43,1,1,1
2,1000817480846862155,43,1,1,1
3,1004943868572044494,43,1,1,1
4,1005164764874054592,43,1,1,1
...,...,...,...,...,...
7087,996135836802743128,43,1,1,1
7088,997545005038320324,43,1,1,1
7089,997691518031628045,43,1,1,0
7090,998529431391212014,43,1,1,1


In [31]:
df_transacoes.isnull().sum()

internal_store_id      0
internal_product_id    0
distributor_id         0
transaction_date       0
reference_date         0
quantity               0
gross_value            0
net_value              0
gross_profit           0
discount               0
taxes                  0
semana                 0
dtype: int64

In [6]:
df_transacoes.head()

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,0.234375
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,0.810000
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,0.405000
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,0.000000
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,2.279758


`internal_store_id` de `df_transacoes` corresponde ao `pdv` da tabela `df_cadastro_pdv`. \
`internal_product_id` de `df_transacoes` corresponde ao `produto` da tabela `df_cadastro_prod`.

In [7]:
df_cadastro_pdv.head()

,pdv,premise,categoria_pdv,zipcode
0,2204965430669363375,On Premise,Mexican Rest,30741
1,5211957289528622910,On Premise,Hotel/Motel,80011
2,9024493554530757353,Off Premise,Convenience,80751
3,8659197371382902429,On Premise,Restaurant,80439
4,1400854873763881130,On Premise,Restaurant,30093


In [8]:
df_cadastro_prod.head()

,produto,categoria,descricao,tipos,label,subcategoria,marca,fabricante
0,69753381296344216,Package,MONDAY NIGHT BREW DON'T CALL IT HOTLANTA 6/4/1...,Package,In&Out,Red Wine,Monday Night Don't Call It Hotlanta,Monday Night Brewing
1,5019314249828979377,Distilled Spirits,BARRELL V907 RED PHONE BOOTH & TOWER,Distilled Spirits,Specialty,Lager / Pilsner,Barrell Bourbon- Private Label,Barrell Craft Spirits
2,4016404282141162328,Distilled Spirits,DRINKWORKS SIMPLY REFRESHING GIN & TONIC 12/4/...,Distilled Spirits,Core,Lager,Drinkworks Simply Refreshing Gin & Tonic,AB Drinkworks
3,6217366559810422145,Distilled Spirits,CENZON REPOSADO 100% AGAVE 6/750ML 80PF,Distilled Spirits,None,Lager,Cenzon Reposado 100% Agave Tequila,Sazerac Spirits
4,7356488787409434558,Draft,REFORMATION STARK 1/4 KEG,Draft,Core,Red Wine,Reformation Stark,Reformation Brewery


Arquivo de previsão no formato CSV ou Parquet, com as seguintes colunas:

| semana | pdv  | produto | quantidade |
|--------|------|---------|------------|
| 1      | 1023 | 123     | 120        |
| 2      | 1045 | 234     | 85         |
| 3      | 1023 | 456     | 110       |

No caso do csv, utilize “;” como caractere separador (exemplo: 1;1023;123;120) e encoding UTF-8.

**semana (número inteiro)**: número da semana (1 a 4 de janeiro/2023) \
**pdv (número inteiro)**: código do ponto de venda \
**produto (número inteiro)**: código do SKU \
**quantidade (número inteiro)**: previsão de vendas


## Organizar dados para obter Y

In [9]:
df_transacoes["semana"] = (
    (pd.to_datetime(df_transacoes["transaction_date"]).dt.dayofyear - 1) // 7
) + 1

In [12]:
df_transacoes.head()

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes,semana
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,0.234375,28
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,0.810000,12
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,0.405000,36
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,0.000000,37
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,2.279758,7


In [13]:
df_transacoes.value_counts(["semana", "transaction_date"]).sort_index().head(30)

semana  transaction_date
1       2022-01-01            170
        2022-01-02           1112
        2022-01-03           8833
        2022-01-04          20934
        2022-01-05          18039
        2022-01-06          17456
        2022-01-07          12957
2       2022-01-08            319
        2022-01-09           1601
        2022-01-10          22159
        2022-01-11          21377
        2022-01-12          19034
        2022-01-13          14901
        2022-01-14          11963
3       2022-01-15            373
        2022-01-16            732
        2022-01-17           8744
        2022-01-18          23937
        2022-01-19          21639
        2022-01-20          18973
        2022-01-21          14709
4       2022-01-22            589
        2022-01-23           2348
        2022-01-24          25546
        2022-01-25          21369
        2022-01-26          19286
        2022-01-27          16072
        2022-01-28          12957
5       2022-01-29     

In [14]:
df_transacoes_grouped = (
    df_transacoes.groupby(["semana", "internal_store_id", "internal_product_id"])
    .agg({"quantity": "sum"})
    .reset_index()
)
df_transacoes_grouped = df_transacoes_grouped.rename(
    columns={
        "internal_store_id": "pdv",
        "internal_product_id": "produto",
        "quantity": "quantidade",
    }
)

In [15]:
df_transacoes_grouped.sort_index().head(20)

,semana,pdv,produto,quantidade
0,1,1001371918471115422,1029370090212151375,2.0
1,1,1001371918471115422,1120490062981954254,18.0
2,1,1001371918471115422,2239307647969388381,2.0
3,1,1001371918471115422,4353552881410365573,7.0
4,1,1001371918471115422,4797439216678436447,1.0
5,1,1001371918471115422,5913249594472983850,7.0
6,1,1001371918471115422,7852170368684115755,15.0
7,1,1001371918471115422,78720985238870578,2.0
8,1,1001371918471115422,8222093130128661995,2.0
9,1,1001371918471115422,8305237228324117368,2.0


In [16]:
df_transacoes

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes,semana
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,0.234375,28
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,0.810000,12
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,0.405000,36
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,0.000000,37
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,2.279758,7
...,...,...,...,...,...,...,...,...,...,...,...,...
6560693,1439579822593789678,2523145330235258825,4,2022-12-06,2022-12-01,3.0,63.000000,61.379761,20.879761,0.000000,1.620240,49
6560694,2357589266242432199,3148628429396585683,4,2022-05-20,2022-05-01,1.0,12.300000,12.300000,3.650000,2.800000,0.000000,20
6560695,7245187752757348055,6668439675743950745,4,2022-08-17,2022-08-01,1.0,33.849998,31.570240,10.191240,0.000000,2.279758,33
6560696,8952609383990245450,3894706280449257667,5,2022-02-03,2022-02-01,4.0,74.556000,73.655998,17.991999,15.600000,0.900000,5


In [20]:
df_cadastro_prod.produto.unique().shape

(7092,)

In [23]:
df_cadastro_prod.drop_duplicates().shape

(304956, 8)

In [ ]:
df_cadastro_prod = df_cadastro_prod.drop_duplicates(subset=["produto"])
df_cadastro_prod.to_parquet(
    "/data/challenges/hackathon_bigdata_2025/processed_data/cadastro_produtos.parquet",
    index=False,
)